# Volatility Surface Interpolation

Given a grid of observed option prices, we can interpolate the volatility (or price) surface across strike $K$ and maturity $T$.  
Different interpolation schemes provide different degrees of smoothness — an important choice when computing derivatives such as $\partial^2 C / \partial K^2$ for the Dupire local volatility formula.

---

### 🔹 Interpolation Methods Summary

| Method | Meaning | Smoothness | Function |
|:--|:--|:--|:--|
| **Linear (kx=1, ky=1)** | Piecewise planar interpolation between grid points. Produces straight-edge surfaces. | $C^0$ (continuous only) | `bisplrep`, `bisplev` |
| **Quadratic (kx=2, ky=2)** | Piecewise quadratic patches with moderate curvature. | $C^1$ (first derivative continuous) | `bisplrep`, `bisplev` |
| **Cubic (kx=3, ky=3)** | Piecewise cubic patches with smooth slope and curvature. | $C^2$ (first & second derivatives continuous) | `bisplrep`, `bisplev` |
| **Bivariate Cubic Spline** | True 2-D spline on a rectangular grid, smooth in both $K$ and $T$. | $C^2$ (globally smooth, stable for PDEs) | `RectBivariateSpline` |


---

- `bisplrep` -> Bi-Spline Representation
- `bisplev` -> Bi-Spline Evaluation

## Market Call Prices

In [1]:
# Call Market Prices

import polars as pl

# --- Spot and curve parameters ---
S0 = 100.0
r = 0.02
q = 0.01

# --- Grid data ---
strikes = [80, 90, 100, 110, 120]
maturities = [0.25, 0.5, 1.0, 2.0]

# --- Call price grid - Assumed to be market-observed ---
prices = [
    [20.23, 21.07, 22.78, 25.41],
    [11.08, 12.36, 14.43, 17.94],
    [3.81,  5.38,  7.97,  11.93],
    [0.92,  1.76,  3.41,  6.20],
    [0.18,  0.47,  1.20,  2.93],
]

# --- Build Polars DataFrame in long format ---
df = (
    pl.DataFrame(prices,
                 schema=[str(t) + "Y" for t in maturities],
                 orient="row")
    # Add a new column called "Strike"
    .with_columns(pl.Series("Strike", strikes))
    # Keep "Strike" fixed as the index (that’s the x-dimension),
    .unpivot(index=["Strike"], 
            # Take all the maturity columns (e.g. "0.25Y", "0.5Y", etc.),
            on=[str(t) + "Y" for t in maturities],
            # Stack them vertically into two new columns:
            # "Maturity" → will hold "0.25Y", "0.5Y", etc.
            # "CallPrice" → will hold the actual numerical prices.
            variable_name="Maturity", value_name="CallPrice")
    # Remove the letter “Y” (so "0.25Y" → "0.25")
    # Convert the column to float (0.25, 0.5, 1.0, 2.0)
    .with_columns(pl.col("Maturity").str.replace("Y", "").cast(pl.Float64))
)
df


Strike,Maturity,CallPrice
i64,f64,f64
80,0.25,20.23
90,0.25,11.08
100,0.25,3.81
110,0.25,0.92
120,0.25,0.18
…,…,…
80,2.0,25.41
90,2.0,17.94
100,2.0,11.93


## Raw Volatility Surface

In [8]:
import plotly.graph_objects as go
import numpy as np

# Prepare data grid for plotting
strikes = df["Strike"].unique().sort().to_numpy()
maturities = df["Maturity"].unique().sort().to_numpy()

# Pivot to 2D matrix of call prices
price_matrix = (
    df.to_pandas()
      .pivot(index="Strike", columns="Maturity", values="CallPrice")
      .sort_index(ascending=True)
      .to_numpy()
)

# Build meshgrid for the surface
T_grid, K_grid = np.meshgrid(maturities, strikes)

# --- Prepare red market data points ---
x_points = df["Maturity"].to_numpy()
y_points = df["Strike"].to_numpy()
z_points = df["CallPrice"].to_numpy()

# Create interactive 3D surface + market dots
fig = go.Figure(data=[
    # Interpolated or gridded surface
    go.Surface(
        x=T_grid,
        y=K_grid,
        z=price_matrix,
        colorscale="Viridis",
        opacity=0.8,
        contours={"z": {"show": True, "usecolormap": True, "highlightcolor": "limegreen"}},
        name="Interpolated Surface",
    ),
    # Actual market points
    go.Scatter3d(
        x=x_points,
        y=y_points,
        z=z_points,
        mode="markers",
        marker=dict(size=5, color="red", symbol="circle"),
        name="Market Data",
    )
])

# Layout and labels
fig.update_layout(
    title="Market Call Price Surface (with Observed Data Points)",
    scene=dict(
        xaxis_title="Maturity (Years)",
        yaxis_title="Strike",
        zaxis_title="Call Price",
    ),
    width=850,
    height=650,
    template="plotly_white",
)

fig.show()


## Interpolated Volatility Surfaces

### 1 - Plotting: Creating a grid for the surface using `np.meshgrid`

When building a volatility or price surface, we often have two independent variables:
- **Strike ($K$)**
- **Maturity ($T$)**  

We want to evaluate (or plot) the function $C(K, T)$ for *every combination* of $K$ and $T$.  
That’s exactly what `np.meshgrid` creates — a full 2D coordinate grid.

#### Example

```python
K_fine = [80, 90, 100, 110]     # 4 strikes
T_fine = [0.5, 1.0, 2.0]        # 3 maturities
K_grid, T_grid = np.meshgrid(K_fine, T_fine)


T_grid = 
[[0.5 0.5 0.5 0.5]
 [1.0 1.0 1.0 1.0]
 [2.0 2.0 2.0 2.0]]

K_grid = 
[[ 80  90 100 110]
 [ 80  90 100 110]
 [ 80  90 100 110]]
```

#### 🔍 Interpretation

Each cell $(i, j)$ in these 2D arrays corresponds to a unique $(K, T)$ pair:

- $(K, T) = (80, 0.5)$ — top-left
- $(K, T) = (110, 2.0)$ — bottom-right

Together, they form all combinations of strikes and maturities needed to evaluate or plot $C(K, T)$.

| Array    | What It Does                                          | Meaning                                |
|:---------|:------------------------------------------------------|:---------------------------------------|
| `K_grid` | Repeats the strike vector `K_fine` once per maturity  | Expands $K$ horizontally across maturities |
| `T_grid` | Repeats the maturity vector `T_fine` across all strikes | Expands $T$ vertically across strikes |

So np.meshgrid doesn’t just concatenate — it expands both coordinate vectors into full 2D matrices that align with your surface function $C(K, T)$ and with 3D plotting tools like go.Surface.

The `K_grid` and `T_grid` are used for plotting i.e. input of `go.Surface`!


### 2- Flattening for `bisplrep` and then usage in `bisplev`

The functions `scipy.interpolate.bisplrep()` and `scipy.interpolate.bisplev()` work **together**:

| Function | Purpose | Input | Output |
|:--|:--|:--|:--|
| `bisplrep(x, y, z, kx, ky)` | Fits a bivariate spline surface to the data | 1D arrays of `x`, `y`, and `z` | A tuple `(t, c, k)` = spline representation |
| `bisplev(x_new, y_new, tck)` | Evaluates the spline at new points | New `x`, `y`, and the tuple `tck` | Interpolated or smoothed `z` values |

---

####  `bisplrep()`:

```python
tck_linear = interpolate.bisplrep(x, y, z, kx=1, ky=1)
```

- t = arrays of knot locations in the $x$ (maturity) and $y$ (strike) directions — basically where the polynomial pieces join.
- c = spline coefficients — these are the weights that determine the value of the surface in each region.
- k = polynomial degree (kx, ky).

Basically:
- `bisplrep` -> fit the surface (creates the model)
- `bisplev` -> evaluate the surface (produces interpolated prices)

In [27]:
import numpy as np
from scipy import interpolate
import plotly.graph_objects as go
from plotly.subplots import make_subplots


# Convert to numpy arrays
strikes = np.array(strikes)
maturities = np.array(maturities)
prices = np.array(prices)

# --------------------------------------------------------------------------
# --- Create a fine meshgrid for interpolation
# 1) Fine grid of evaluation points --> 100×100 grid for surface
# This are going to be used in `bisplev`
K_fine = np.linspace(strikes.min(), strikes.max(), 100)
T_fine = np.linspace(maturities.min(), maturities.max(), 100)

# 2) Build 2D matrices of coordinates
# These are going to be used in `go.Surface`
K_grid, T_grid = np.meshgrid(K_fine, T_fine)

# --------------------------------------------------------------------------
# 3) Flatten for `bisplrep`` (requires 1D x,y,z arrays) --- Used also for the red dot
T_flat, K_flat = np.meshgrid(maturities, strikes)   # maturity first, strike second
x = T_flat.ravel()   # maturities
y = K_flat.ravel()   # strikes
z = prices.ravel()   # call prices

# --------------------------------------------------------------------------
# 4) Interpolations
# The output tck_linear contains the fitted spline coefficients (a “spline representation”).
tck_linear = interpolate.bisplrep(x, y, z, kx=1, ky=1)
# evaluates the fitted surface on the fine grid (T_fine, K_fine) created earlier.
C_linear = interpolate.bisplev(T_fine, K_fine, tck_linear)

tck_quadratic = interpolate.bisplrep(x, y, z, kx=2, ky=2)
C_quadratic = interpolate.bisplev(T_fine, K_fine, tck_quadratic)

tck_cubic = interpolate.bisplrep(x, y, z, kx=3, ky=3)
C_cubic = interpolate.bisplev(T_fine, K_fine, tck_cubic)

bivar_cubic = interpolate.RectBivariateSpline(maturities,strikes, prices.T) # transpose Price to match surface orientation
C_bivar = bivar_cubic(T_fine,K_fine)  


# Combine in one 2x2 subplot
fig = make_subplots(
    rows=2, cols=2,
    specs=[[{"type": "surface"}, {"type": "surface"}],
           [{"type": "surface"}, {"type": "surface"}]],
    subplot_titles=(
        "Linear Surface (kx=1, ky=1)",
        "Quadratic Surface (kx=2, ky=2)",
        "Cubic Surface (kx=3, ky=3)",
        "RectBivariateSpline Surface"
    )
)

# Helper to add each surface
def add_surface(fig, z_surface, row, col, colorscale):
    fig.add_trace(
        go.Surface(
            x=T_grid, y=K_grid, z=z_surface,
            colorscale=colorscale, opacity=0.85, showscale=False
        ),
        row=row, col=col
    )
    fig.add_trace(
      go.Scatter3d(
            x=x, y=y, z=z,
            mode="markers",
            marker=dict(size=3, color="red"),
            name="Market data",
            showlegend=(row == 1 and col == 1)
        ),
        row=row, col=col
    )

# Add each surface to the figure
add_surface(fig, C_linear, 1, 1, "Blues")
add_surface(fig, C_quadratic, 1, 2, "Purples")
add_surface(fig, C_cubic, 2, 1, "Greens")
add_surface(fig, C_bivar, 2, 2, "Viridis")

# Layout
fig.update_layout(
    title_text="Implied Volatility Surface Interpolations",
    width=1500,   # wider figure
    height=1100,  # taller figure
    template="plotly_white",
    margin=dict(l=40, r=40, b=40, t=80),  
    scene=dict(
        domain=dict(x=[0.0, 0.48], y=[0.52, 1.0]),
        xaxis_title="Maturity (Years)",
        yaxis_title="Strike",
        zaxis_title="Call Price",
    ),
    scene2=dict(
        domain=dict(x=[0.52, 1.0], y=[0.52, 1.0]),
        xaxis_title="Maturity (Years)",
        yaxis_title="Strike",
        zaxis_title="Call Price",
    ),
    scene3=dict(
        domain=dict(x=[0.0, 0.48], y=[0.0, 0.48]),
        xaxis_title="Maturity (Years)",
        yaxis_title="Strike",
        zaxis_title="Call Price",
    ),
    scene4=dict(
        domain=dict(x=[0.52, 1.0], y=[0.0, 0.48]),
        xaxis_title="Maturity (Years)",
        yaxis_title="Strike",
        zaxis_title="Call Price",
    ),
)

fig.show()

In [25]:
fig.layout.scene3.camera

layout.scene.Camera()